In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import os
import nltk
import string
import numpy as np
import pandas as pd
from natsort import natsorted

### Reading and Preprocessing Dataset:

In [46]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/IR_assign2/Humor,Hist,Media,Food.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [47]:
file_names = natsorted(os.listdir('/content/Humor,Hist,Media,Food'))
dat= []

for file_name in file_names:
    path = '/content/Humor,Hist,Media,Food/'+file_name
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        temp = {}
        temp['name'] = file_name
        temp['text'] = f.read()
        dat.append(temp)

df = pd.DataFrame(dat)
df.head()

name                                               text
0   1st_aid.txt  HERBALHERB1ST AIDCALENDULACOMFREYREMEDIESSICKM...
1        a-team  From uunet!cs.utexas.edu!usc!ucsd!ucbvax!CAE.W...
2  a_fish_c.apo  From: murph@buscard.fidonet.org (Brian Murphy)...
3  a_tv_t-p.com  ____________________________________________\n...
4    abbott.txt  \n                              Abbott & Coste...

In [48]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
import re
import numpy as np
from tqdm import tqdm
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

cachedStopWords = stopwords.words("english")

In [50]:
# References: https://williamscott701.medium.com/information-retrieval-unigram-postings-and-positional-postings-a28b907c4e8
chedStopWords = stopwords.words("english")
def rm_stopwords(dat):
    words = word_tokenize(str(dat))
    lis=[]
    for word in words:
      if(word not in chedStopWords):
        lis.append(word)
    res=' '.join(lis)
    return np.char.strip(res)

def removepunct(dat):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    i=0
    while i<len(punc):
        dat= np.char.replace(dat, punc[i], ' ')
        dat= np.char.replace(dat, "  ", " ")
        i=i+1
    dat= np.char.replace(dat, ',', '')
    dat=np.char.replace(dat,'!','')
    return dat


def lemmatization(dat):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(str(dat))
    new_text = ""
    for w in tokens:
        new_text = new_text + " "
        new_text=new_text+ lemmatizer.lemmatize(w)
    return np.char.strip(new_text)

def lowercase(dat):
    return np.char.lower(dat)

def preprocess(dat):
    dat= lowercase(dat)
    dat= removepunct(dat)
    dat= rm_stopwords(dat)
    dat= lemmatization(dat)
    return dat

In [51]:
def clean_text(data_df):
    for index, row in tqdm(data_df.iterrows(), total=data_df.shape[0]):
        sample = row['text']
        data_df.loc[index, 'text'] = str(preprocess(sample))
    return data_df

df = clean_text(df.copy())
df.head()

100%|██████████| 1133/1133 [00:25<00:00, 44.44it/s]


name                                               text
0   1st_aid.txt  herbalherb1st aidcalendulacomfreyremediessickm...
1        a-team  uunet c utexas edu usc ucsd ucbvax cae wisc ed...
2  a_fish_c.apo  murph buscard fidonet org brian murphy subject...
3  a_tv_t-p.com  survey result computer use fan alt tv twin pea...
4    abbott.txt  abbott costello first abbott well costello goi...

## First Part:

In [52]:
from textblob import TextBlob

def jaccard(list1, list2):
    fin=list(set(list1).intersection(list2))
    intersection = len(fin)
    us = (len(list1) + len(list2)) 
    union=us- intersection
    ans=float(intersection)/union
    return ans

def gettopfive(query, df):
    alldoc=[]
    for i in df['text']:
      alldoc.append(TextBlob(i))

    # alldocs = [TextBlob(i) for i in df['text']]
    
    cleaned_query = TextBlob(str(preprocess(query)))
    scores = []
    # for i in (alldoc):
    i=0
    while i <len(alldoc):
        score = jaccard(list(alldoc[i].words), list(cleaned_query.words))
        scores.append(score)
        i+=1

    temp = pd.DataFrame()
    temp['doc'] = df['name']
    temp['score'] = scores

    return temp.sort_values(by=['score'], ascending=False).reset_index(drop=True).head(5)

In [53]:
query = 'Effective against some common internal parasites if anyone has any additions or corrections'

gettopfive(query, df)

doc     score
0      flowchrt  0.033333
1  flowchrt.txt  0.017857
2   1st_aid.txt  0.017467
3       popmach  0.010204
4    odd_to.obs  0.009967

## Second Part:

In [54]:
from textblob import TextBlob
import math

def tf_rawcount(word, counter):
    anstoret=counter[word]
    return anstoret
def tf(word, counter):
    totallength=len(counter)
    return counter[word] / totallength

def tf_binary(word, counter):
    return int(tf_rawcount(word,counter)>0)

def tf_lognorm(word, counter):
    return math.log(1+tf_rawcount(word, counter))

def idf(word, postings):
    lenpost=len(postings)
    postget=postings[word]
    return math.log(lenpost / (1 + postget))

def get_corpus(df):
    tojoin=df['text']
    all_text = TextBlob(' '.join(tojoin))
    templisty=list(all_text.words)
    counter=collections.Counter(templisty)
    
    index = 0
    for key, value in counter.items():
        counter[key] = -1*index
        index-=1
    
    return counter

def stringer(x):
    return str(x)

In [55]:
df.head()

name                                               text
0   1st_aid.txt  herbalherb1st aidcalendulacomfreyremediessickm...
1        a-team  uunet c utexas edu usc ucsd ucbvax cae wisc ed...
2  a_fish_c.apo  murph buscard fidonet org brian murphy subject...
3  a_tv_t-p.com  survey result computer use fan alt tv twin pea...
4    abbott.txt  abbott costello first abbott well costello goi...

In [56]:
import collections

for i in range(5):
  if(i==1):
    vect1=[]
  elif(i==2):
    vect2=[]
  elif(i==3):
    vect3=[]
  elif(i==4):
    vect4=[]
  else:
    vect5=[]
all_text=[]
for i in df['text']:
  all_text.append(TextBlob(i))


postings = {}

for index, row in df.iterrows():
    tokens = TextBlob(str(row['text'])).words
    for token in tokens:
        if token in postings:
            postings[token] += 1
        else:
            postings[token] = 0

corpus = get_corpus(df)

for i, blob in (enumerate(all_text)):
    vec1 = np.zeros((len(corpus),))
    vec2 = np.zeros((len(corpus),))
    vec3 = np.zeros((len(corpus),))
    vec4 = np.zeros((len(corpus),))
    vec5 = np.zeros((len(corpus),))
    counter=collections.Counter(list(blob.words))

    for word in blob.words:
        vec1[corpus[word]] = tf(word, counter) * idf(word, postings)
        vec2[corpus[word]] = tf_binary(word, counter) * idf(word, postings)
        vec3[corpus[word]] = tf_rawcount(word, counter) * idf(word, postings)
        vec4[corpus[word]] = tf_lognorm(word, counter) * idf(word, postings)
    
    vect1.append(vec1)
    vect2.append(vec2)
    vect3.append(vec3)
    vect4.append(vec4)
    vect5.append(vec5)

In [57]:
# Garbage Collector - use it like gc.collect()
import gc
gc.collect()

50

In [58]:
vect1 = np.array(vect1)
vect2 = np.array(vect2)
vect3 = np.array(vect3)
vect4 = np.array(vect4)
vect5 = np.array(vect5)

In [59]:
query = 'Suitable for all bruises and minor damage to external blood vessels and veins'
cleaned_query = TextBlob(str(preprocess(query)))

In [60]:
binary = [0]
raw_count = [0]
term_frequency = [0]
log_normalization = [0]
double_normalization = [0]
j=0
while(j<5):
  if(j==1):
    binary = [0]*len(df)
  elif(j==2):
    raw_count = [0]*len(df)
  elif(j==3):
    term_frequency = [0]*len(df)
  elif(j==4):
    log_normalization = [0]*len(df)
  else:
    double_normalization = [0]*len(df)
  j+=1

def funx(vecs,i,corpus):
    tel=(np.array(vecs)[:, corpus[i]])
    return tel

for i in list(cleaned_query.words):
    ff=funx(vect1,i,corpus)
    binary+=ff

for i in list(cleaned_query.words):
    ff=funx(vect2,i,corpus)
    raw_count+=ff

for i in list(cleaned_query.words):
    ff=funx(vect3,i,corpus)
    term_frequency+=ff

for i in list(cleaned_query.words):
    ff=funx(vect4,i,corpus)
    log_normalization+=ff

for i in list(cleaned_query.words):
    ff=funx(vect5,i,corpus)
    double_normalization+=ff

In [61]:
cases=[1,2,3,4]
for case in cases:
  temp_dict={}
  counter=0
  if(case==1):
    for i in df['name']:
      temp_dict[i] = binary[counter]
      counter+=1
    print("binary_dict:")
  elif(case==2):
    for i in df['name']:
      temp_dict[i] = raw_count[counter]
      counter+=1
    print("raw_count:")
  elif(case==3):
    for i in df['name']:
      temp_dict[i] = term_frequency[counter]
      counter+=1
    print("term_frequency:")
  else:
    for i in df['name']:
      temp_dict[i] = log_normalization[counter]
      counter+=1
    print("log_normalization:")
  print(sorted(temp_dict.items(), key=lambda x: x[1], reverse=True)[:5])
  


binary_dict:
[('1st_aid.txt', 0.5561427221359942), ('letgosh.txt', 0.2408049604946773), ('blood.txt', 0.22084109077296685), ('let.go', 0.20816387889309704), ('hangover.txt', 0.20323182636366194)]
raw_count:
[('1st_aid.txt', 56.130321814164745), ('bnbeg2.4.txt', 27.605589006654093), ('bnbguide.txt', 27.605589006654093), ('epi_bnb.txt', 27.605589006654093), ('bhb.ill', 26.912441826094152)]
term_frequency:
[('hackingcracking.txt', 112.90551381358489), ('bhb.ill', 89.98735995260765), ('butcher.txt', 89.98735995260765), ('1st_aid.txt', 84.53369376467116), ('let.go', 81.39207664720094)]
log_normalization:
[('1st_aid.txt', 49.61938257778032), ('bhb.ill', 34.8932622261153), ('butcher.txt', 34.8932622261153), ('hackingcracking.txt', 30.135218353092544), ('practica.txt', 29.73105960338742)]
